In [3]:
from sqlite3 import connect

# from datetime import datetime
import json
from webbrowser import get

# from requests.auth import HTTPBasicAuth
import configuration_class
import subprocess
import os 
import sys
import shutil
import logging
from io import StringIO

import datetime as dt
import time

import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from airflow.hooks.base_hook import BaseHook
# to last_success_date_epoch needen
from dateutil.parser import *   
import calendar

In [4]:
monitoring_failde_pods = configuration_class.OpenShiftPodsMonitoring()
monitoring_failed_pods_data_pipeline = monitoring_failde_pods.MONITORING_FAILED_PODS_DATA_PIPELINE_MONITORING_PROD_DICT

In [5]:
def difference_dates_hours(date1):
    now = dt.datetime.now()
    beauty_time = now.strftime("%Y-%m-%d  %H:%M:%S")
    beauty_time = dt.datetime.strptime(beauty_time,"%Y-%m-%d  %H:%M:%S")
    delta = (beauty_time - date1).total_seconds()
    min = delta / 60
    hours = delta / (60 * 60) 
    # print( f''' JESTEM W differences_dates_hours now = {type(beauty_time) } - {beauty_time}  DATE1 = {type(date1)} , {date1} 
    #       \n delta = {type(delta)}   {delta}  min = {min}   hours = {hours}''')
    
    return round(hours,2)
   
def difference_dates_minutes(date1 ):
    now = dt.datetime.now()
    beauty_time = now.strftime("%Y-%m-%d  %H:%M:%S")
    beauty_time = dt.datetime.strptime(beauty_time,"%Y-%m-%d  %H:%M:%S")
    delta = (beauty_time - date1).total_seconds()
    min = delta / 60 
    # hours = delta / (60 * 60) 
    # print( f''' JESTEM W differences_dates_hours now = {type(beauty_time) } - {beauty_time}  DATE1 = {type(date1)} , {date1} 
    #       \n delta = {type(delta)}   {delta}  min = {min}   hours = {hours}''')
    
    return round(min,2)
    
def difference_dates_seconds(date1):
    now = dt.now()
    beauty_time = now.strftime("%Y-%m-%d  %H:%M:%S")
    beauty_time = dt.datetime.strptime(beauty_time,"%Y-%m-%d  %H:%M:%S")
    delta = (beauty_time - date1).total_seconds()
    # min = delta / 60
    # hours = delta / (60 * 60) 
    # print( f''' JESTEM W differences_dates_hours now = {type(beauty_time) } - {beauty_time}  DATE1 = {type(date1)} , {date1} 
    #       \n delta = {type(delta)}   {delta}  min = {min}   hours = {hours}''')
    
    return delta
def change_creationtimestamp(date):
        date = dt.datetime.fromisoformat(date[:-1] + '+00:00')
        new_time = date + dt.timedelta(hours=2) 
        new_time = new_time.strftime("%Y-%m-%d  %H:%M:%S")
        return dt.datetime.strptime(new_time,"%Y-%m-%d  %H:%M:%S")
    


In [6]:
def date_to_epoch(date_input, in_seconds=False):
    ''' 
    INPUT: date as a datetime object, or as a string - then formats accepted are like '2022-07-07T08:00:00', '2022-07-07T08:00:00+00:00', '2022-07-07T08:00:00-02:00', '2022-07-07 08:00:00,100', '2022-07-07T08:00:00.100z', 'Jul 7, 2022 @ 08:00:00.683' etc.
    IMPORTANT: if input date is NAIVE, it is treated like it is in UTC
    OUTPUT: unix timestamp (epoch time). Default in miliseconds, in seconds if the second parameter = True
    '''

    print('date string to epoch converter:')
    print('input: '+str(date_input))

    if type(date_input) == str:
        if '@' in date_input:
            date_input = date_input.replace("@", "")
        date = parse(date_input)
    else:
        date = date_input

    epoch = calendar.timegm(date.utctimetuple())
    print(dt.datetime.utcfromtimestamp(epoch).strftime('%Y-%m-%d %H:%M:%S'))
    if in_seconds == False:
        epoch = epoch*1000
    print('date to epoch converter return: '+str(epoch))
    return epoch


print( date_to_epoch('2022-08-17T07:40:00Z'))
print( date_to_epoch('2022-09-13T14:13:42Z'))

a = date_to_epoch('2022-08-17T07:40:00', True )
b = dt.datetime.fromtimestamp(a) 
print("Unix_Time =>",a)
print("Date & Time =>" ,    type( b.strftime('%Y-%m-%d %H:%M:%S')))

date string to epoch converter:
input: 2022-08-17T07:40:00Z
2022-08-17 07:40:00
date to epoch converter return: 1660722000000
1660722000000
date string to epoch converter:
input: 2022-09-13T14:13:42Z
2022-09-13 14:13:42
date to epoch converter return: 1663078422000
1663078422000
date string to epoch converter:
input: 2022-08-17T07:40:00
2022-08-17 07:40:00
date to epoch converter return: 1660722000
Unix_Time => 1660722000
Date & Time => <class 'str'>


In [8]:
def get_grafana_dashboards():
    TOKEN = "eyJrIjoiejdFN3hSZ3pva2kwYVE3Mmg0ek1Gc3dxNlh0OWhlUUQiLCJuIjoicmFkZWsiLCJpZCI6Mn0="
    connection = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        "token" : TOKEN,
        "headers" : {'Accept': 'application/json',"Authorization": "Bearer "+ TOKEN},
        "link"    : "https://grafana-centralized-monitoring.advantagedp.org/api/search",
        "link0"   : "https://grafana-centralized-monitoring.advantagedp.org/api/folders",
        "link_db" : "https://grafana-centralized-monitoring.advantagedp.org/api/folders/ZsM-EZyZk", # folder
        "linkf"   : "https://grafana-centralized-monitoring.advantagedp.org/api/folders/id/778",
        "link_search" : "https://grafana-centralized-monitoring.advantagedp.org/api/search?query=&",
        "link_json"   : "https://grafana-centralized-monitoring.advantagedp.org/api/dashboards/uid/"
  

    }
#     mkdir -p dashboards && for dash in $(curl -k -H "Authorization: Bearer $KEY" $HOST/api/search\?query\=\& |tr ']' '\n' |cut -d "," -f 5 |grep slug |cut -d\" -f 4); do 
#   curl -k -H "Authorization: Bearer $KEY" $HOST/api/dashboards/db/$dash > dashboards/$dash.json 
# done
    res = requests.get(connection['link'], headers=connection['headers'], proxies=connection['proxies'], verify=False)
    folders = requests.get(connection['link0'], headers=connection['headers'], proxies=connection['proxies'], verify=False)
    dashboards = requests.get(connection['link_db'], headers=connection['headers'], proxies=connection['proxies'], verify=False)
    get_forder_by_id = requests.get(connection['linkf'], headers=connection['headers'], proxies=connection['proxies'], verify=False)

    print( 'res ',res )
    print( 'folders ',folders )
    print( 'dashboards ',dashboards )
    print( 'get_forder_by_id ',get_forder_by_id )

    res = res.json()
    folders = folders.json()
    dashboards = dashboards.json()
    get_forder_by_id = get_forder_by_id.json()

    # res_db = res_db.json()
    # print( f'res_db 0     {res_db}')
    print( f'{len(res)} , \n  {res[0]} ')
    print( f' keys of res {res[0].keys()}')
    print( f'{len(folders)} , \n  {folders[0:3]} ')
    print( f' keys of res {folders[0].keys()}')
    print( f'\n \n ---- \n \n {type(dashboards)} , \n  {dashboards} ')
    print( f' keys of res {dashboards.keys()}')
    # for key , item in dashboards.items():
    #     print(f' klucz {key} --  {dashboards[key]}')
    # print( f' \n  ==== \n {type(get_forder_by_id)} , \n  {get_forder_by_id.keys()}')
    # for key , item in get_forder_by_id.items():
    #     print(f' klucz {key} --  {get_forder_by_id[key]}')

    id,uid, title, uri,url, slug, type_ ,tags, isStarred, sortMeta = zip(# title, uri, url, slug, type, tags, isStarred, sortMeta = zip(
        *(
            (
                item["id"],
                item["uid"],
                item["title"],
                item["uri"],
                item["url"],
                item["slug"],
                item["type"],
                item["tags"],
                item["isStarred"],
                item["sortMeta"],
                
            )
            for item in res
        )
    )
    df = pd.DataFrame(
        {
            "Id": id,
            "Uid": uid,
            "Title": title,
            "Uri": uri,
            "Url": url,
            "Slug": slug,
            "Type": type_,
            "Tags": tags,
            "IsStarred": isStarred,
            "SortMeta": sortMeta,
        }
    )
    # print(  df[0:15:2] ) 
    # print( df.iloc[12])
    # boson = df.iloc[12]
    return df
all_dashboardy = get_grafana_dashboards()    
boson_dashboards = all_dashboardy.loc[12]


/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

res  <Response [200]>
folders  <Response [200]>
dashboards  <Response [200]>
get_forder_by_id  <Response [200]>
1000 , 
  {'id': 6221, 'uid': 'PxXOwGa7k', 'title': 'ACC', 'uri': 'db/acc', 'url': '/dashboards/f/PxXOwGa7k/acc', 'slug': '', 'type': 'dash-folder', 'tags': [], 'isStarred': False, 'sortMeta': 0} 
 keys of res dict_keys(['id', 'uid', 'title', 'uri', 'url', 'slug', 'type', 'tags', 'isStarred', 'sortMeta'])
85 , 
  [{'id': 6221, 'uid': 'PxXOwGa7k', 'title': 'ACC'}, {'id': 675, 'uid': 't39yCXxZk', 'title': 'AD-Labeling'}, {'id': 988, 'uid': 'EHNfu23Zk', 'title': 'Airflow'}] 
 keys of res dict_keys(['id', 'uid', 'title'])

 
 ---- 
 
 <class 'dict'> , 
  {'id': 778, 'uid': 'ZsM-EZyZk', 'title': 'Boson', 'url': '/dashboards/f/ZsM-EZyZk/boson', 'hasAcl': False, 'canSave': False, 'canEdit': False, 'canAdmin': False, 'createdBy': 'admin', 'created': '2020-01-20T07:40:36Z', 'updatedBy': 'bartlomiej.grzeskowiak@dxc.com', 'updated': '2021-12-02T14:20:36Z', 'version': 2} 
 keys of res di

In [9]:
today = dt.datetime.now()
print( today.year, today.month, str(today.day))

2022 10 19


In [17]:
def get_grafana_dashboards_backups():
    TOKEN = "eyJrIjoiejdFN3hSZ3pva2kwYVE3Mmg0ek1Gc3dxNlh0OWhlUUQiLCJuIjoicmFkZWsiLCJpZCI6Mn0="
    connection = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        "token" : TOKEN,
        "headers" : {'Accept': 'application/json',"Authorization": "Bearer "+ TOKEN},
        "link"    : "https://grafana-centralized-monitoring.advantagedp.org/api/search",
        "linkfolders"   : "https://grafana-centralized-monitoring.advantagedp.org/api/folders",
        "link_db" : "https://grafana-centralized-monitoring.advantagedp.org/api/folders/ZsM-EZyZk", # folder
        "linkf"   : "https://grafana-centralized-monitoring.advantagedp.org/api/folders/id/778",
        "link_search" : "https://grafana-centralized-monitoring.advantagedp.org/api/search?query=&",
        "link_json"   : "https://grafana-centralized-monitoring.advantagedp.org/api/dashboards/uid/"
  

    }

#     mkdir -p dashboards && for dash in $(curl -k -H "Authorization: Bearer $KEY" $HOST/api/search\?query\=\& |tr ']' '\n' |cut -d "," -f 5 |grep slug |cut -d\" -f 4); do 
#   curl -k -H "Authorization: Bearer $KEY" $HOST/api/dashboards/db/$dash > dashboards/$dash.json 
# done
    res = requests.get(connection['link'], headers=connection['headers'], proxies=connection['proxies'], verify=False)
    folders = requests.get(connection['linkfolders'], headers=connection['headers'], proxies=connection['proxies'], verify=False)
    # print( 'res ',res )
    # print( 'folders ',folders )

    res = res.json()
    folders = folders.json()
    # print( folders )
    idF,uidF, titleF, = zip(
        *(
            (
                item["id"],
                item["uid"],
                item["title"],                
            )
            for item in folders
        )
    )
    df_folders = pd.DataFrame(
        {
            "Id": idF,
            "Uid": uidF,
            "Title": titleF,

        }
    )
    
    # print(f' \n {df_folders[0:10]} \n ')
    id,uid, title, uri,url, slug, type_ ,tags, isStarred,  sortMeta = zip(#folderId,folderUid, folderTitle, folderUrl
        *(
            (
                item["id"],
                item["uid"],
                item["title"],
                item["uri"],
                item["url"],
                item["slug"],
                item["type"],
                item["tags"],
                item["isStarred"],       
                item["sortMeta"],
                
            )
            for item in res
        )
    )
    df = pd.DataFrame(
        {
            "Id": id,
            "Uid": uid,
            "Title": title,
            "Uri": uri,
            "Url": url,
            "Slug": slug,
            "Type": type_,
            "Tags": tags,
            "IsStarred": isStarred,
            "SortMeta": sortMeta,
        }
    )
    # print(  df[0:15:2] ) 
    # print( df.iloc[12])
    boson = df.iloc[12]
    df['Boson'] = df['Tags'].apply( lambda x : True if 'boson' in ''.join(x).lower() else False )
    boson = df[df['Boson']==True]
    def make_link(text , link = connection['link_json']):
        return link + text
    dir_path = 'Boson'
    today = dt.datetime.now()
    print( today.year, today.month, str(today.day))
    
    dir_path = f'{dir_path}_{str(today.year)}_{str(today.month)}_{str(today.day)}'

    def make_files(name, link ):
        tresc = requests.get(link, headers=connection['headers'], proxies=connection['proxies'], verify=False)
        tresc = tresc.json()
        name = name + '.json'
        with open(name , 'w' , encoding='utf-8') as file:
            json.dump( tresc , file)
        shutil.copy(name,dir_path)
    boson['Name'] = boson['Title'].apply( lambda x : x.replace(" ","_")) 
    boson['Links'] = boson['Uid'].apply( make_link)
    dir_path = 'Boson'
    today = dt.datetime.now()
    print( today.year, today.month, str(today.day))
    
    dir_path = f'{dir_path}_{str(today.year)}_{str(today.month)}_{str(today.day)}'
    list_directories = os.listdir()
    print( list_directories)
    if dir_path  in list_directories:
        print( f' DIRECTORIE {dir_path} EXISTS ')
    else:
        os.mkdir(dir_path)
    # print( boson.head() )
    for ind in boson.index:
        # print( boson['Name'][ind] , boson['Links'][ind] )
        make_files(boson['Name'][ind], boson['Links'][ind]  )
        

    return df , df_folders , boson
all_backups, foldery , boson = get_grafana_dashboards_backups()[0]  ,   get_grafana_dashboards_backups()[1] ,get_grafana_dashboards_backups()[2]
boson_dashboards = all_dashboardy.loc[12]

/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


2022 10 19


/tmp/ipykernel_1473/1691221298.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boson['Name'] = boson['Title'].apply( lambda x : x.replace(" ","_"))
/tmp/ipykernel_1473/1691221298.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boson['Links'] = boson['Uid'].apply( make_link)


2022 10 19
['15_37_plik.txt', 'pody_datapipeline.json', 'res', 'Monitoring_for_Data_Pipeline.json', 'a.py', 'Monitoring_for_Ingest_Service_application.json', 'Monitoring_for_failed_pods_OCP.json', '__pycache__', 'aaa.json', '15_37_condola2.txt', 'configuracja.py', 'Boson_2022_10_19', 'Monitoring_for_Copy_Processes.json', 'restAPI_exercise.ipynb:Zone.Identifier', 'restapi_FAILDE_PODS.ipynb', 'restapi_ALL.ipynb', 'a.ipynb', 'restAPI_exercise.ipynb', 'reingest.txt', 'configuration_class.py', 'restapi_FAILED_PODS.py', 'api1.py', 'bbbb.txt', 'grafa.json', 'Monitoring_for_Copy_Processes_Copy.json', 'Monitoring_for_Session_Checker_application.json', '.vscode', 'Monitoring_for_FOT_application.json', '15_37_konsola.txt', 'bbbb.json', 'restapi_openshift.ipynb', 'Boson_2022_10_18', 'KPI_806.json', 'reingest2.txt', 'rest_api1.ipynb']
 DIRECTORIE Boson_2022_10_19 EXISTS 


/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

...

/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

2022 10 19


/tmp/ipykernel_1473/1691221298.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boson['Name'] = boson['Title'].apply( lambda x : x.replace(" ","_"))
/tmp/ipykernel_1473/1691221298.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boson['Links'] = boson['Uid'].apply( make_link)


2022 10 19
['15_37_plik.txt', 'pody_datapipeline.json', 'res', 'Monitoring_for_Data_Pipeline.json', 'a.py', 'Monitoring_for_Ingest_Service_application.json', 'Monitoring_for_failed_pods_OCP.json', '__pycache__', 'aaa.json', '15_37_condola2.txt', 'configuracja.py', 'Boson_2022_10_19', 'Monitoring_for_Copy_Processes.json', 'restAPI_exercise.ipynb:Zone.Identifier', 'restapi_FAILDE_PODS.ipynb', 'restapi_ALL.ipynb', 'a.ipynb', 'restAPI_exercise.ipynb', 'reingest.txt', 'configuration_class.py', 'restapi_FAILED_PODS.py', 'api1.py', 'bbbb.txt', 'grafa.json', 'Monitoring_for_Copy_Processes_Copy.json', 'Monitoring_for_Session_Checker_application.json', '.vscode', 'Monitoring_for_FOT_application.json', '15_37_konsola.txt', 'bbbb.json', 'restapi_openshift.ipynb', 'Boson_2022_10_18', 'KPI_806.json', 'reingest2.txt', 'rest_api1.ipynb']
 DIRECTORIE Boson_2022_10_19 EXISTS 


/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

In [84]:
def get_grafana_dashboards_backups_json():
    TOKEN = "eyJrIjoiejdFN3hSZ3pva2kwYVE3Mmg0ek1Gc3dxNlh0OWhlUUQiLCJuIjoicmFkZWsiLCJpZCI6Mn0="
    connection = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        "token" : TOKEN,
        "headers" : {'Accept': 'application/json',"Authorization": "Bearer "+ TOKEN},
        "link"    : "https://grafana-centralized-monitoring.advantagedp.org/api/search",
        "linkfolders"   : "https://grafana-centralized-monitoring.advantagedp.org/api/folders",
        "link_db" : "https://grafana-centralized-monitoring.advantagedp.org/api/folders/ZsM-EZyZk", # folder
        "linkf"   : "https://grafana-centralized-monitoring.advantagedp.org/api/folders/id/778",
        "link_search" : "https://grafana-centralized-monitoring.advantagedp.org/api/search?query=&",
        "link_json"   : "https://grafana-centralized-monitoring.advantagedp.org/api/dashboards/uid/"
    }
    def make_link(text , link = connection['link_json']):
        return link + text
    
    def make_files(name, link ):
        tresc = requests.get(link, headers=connection['headers'], proxies=connection['proxies'], verify=False)
        tresc = tresc.json()
        with open(name + '.json', 'w' , encoding='utf-8') as file:
            json.dump( tresc , file)
            
    boson['Name'] = boson['Title'].apply( lambda x : x.replace(" ","_")) 
    boson['Links'] = boson['Uid'].apply( make_link)
    # print( boson.head() )
    for ind in boson.index:
        # print( boson['Name'][ind] , boson['Links'][ind] )
        make_files(boson['Name'][ind], boson['Links'][ind]  )
    return boson
    # res = requests.get(connection['link_json'], headers=connection['headers'], proxies=connection['proxies'], verify=False)
boson1 = get_grafana_dashboards_backups_json()
# for ind in boson.index:
#     print( boson['Title'][ind])
# print( boson1.iloc[0]['Links'])

       Id        Uid                               Title  \
493  1927  3Fu2YiQMk                             KPI 806   
591  6401  TDuJs5Lnz       Monitoring for Copy Processes   
592  6404  EGLqX5Lnk  Monitoring for Copy Processes Copy   
593  2786  44lDUgD7z        Monitoring for Data Pipeline   
595   435  iWqb0G4Vk      Monitoring for failed pods OCP   

                                       Uri  \
493                             db/kpi-806   
591       db/monitoring-for-copy-processes   
592  db/monitoring-for-copy-processes-copy   
593        db/monitoring-for-data-pipeline   
595      db/monitoring-for-failed-pods-ocp   

                                                 Url Slug     Type     Tags  \
493                             /d/3Fu2YiQMk/kpi-806       dash-db  [Boson]   
591       /d/TDuJs5Lnz/monitoring-for-copy-processes       dash-db  [Boson]   
592  /d/EGLqX5Lnk/monitoring-for-copy-processes-copy       dash-db  [boson]   
593        /d/44lDUgD7z/monitoring-for-data-pi

/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Monitoring_for_Copy_Processes https://grafana-centralized-monitoring.advantagedp.org/api/dashboards/uid/TDuJs5Lnz


/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Monitoring_for_Copy_Processes_Copy https://grafana-centralized-monitoring.advantagedp.org/api/dashboards/uid/EGLqX5Lnk


...

/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Monitoring_for_Ingest_Service_application https://grafana-centralized-monitoring.advantagedp.org/api/dashboards/uid/vJu8sgDnk


/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Monitoring_for_Session_Checker_application https://grafana-centralized-monitoring.advantagedp.org/api/dashboards/uid/Okh3Z3H7k


/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'grafana-centralized-monitoring.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [60]:
# boson = all_backups[all_backups['Boson']==True]
# boson

,Id,Uid,Title,Uri,Url,Slug,Type,Tags,IsStarred,SortMeta,Boson
493,1927,3Fu2YiQMk,KPI 806,db/kpi-806,/d/3Fu2YiQMk/kpi-806,,dash-db,[Boson],False,0,True
591,6401,TDuJs5Lnz,Monitoring for Copy Processes,db/monitoring-for-copy-processes,/d/TDuJs5Lnz/monitoring-for-copy-processes,,dash-db,[Boson],False,0,True
592,6404,EGLqX5Lnk,Monitoring for Copy Processes Copy,db/monitoring-for-copy-processes-copy,/d/EGLqX5Lnk/monitoring-for-copy-processes-copy,,dash-db,[boson],False,0,True
593,2786,44lDUgD7z,Monitoring for Data Pipeline,db/monitoring-for-data-pipeline,/d/44lDUgD7z/monitoring-for-data-pipeline,,dash-db,[Boson],False,0,True
595,435,iWqb0G4Vk,Monitoring for failed pods OCP,db/monitoring-for-failed-pods-ocp,/d/iWqb0G4Vk/monitoring-for-failed-pods-ocp,,dash-db,[Boson],False,0,True
596,6389,A4-k-NYnk,Monitoring for FOT application,db/monitoring-for-fot-application,/d/A4-k-NYnk/monitoring-for-fot-application,,dash-db,[boson],False,0,True
597,2783,vJu8sgDnk,Monitoring for Ingest Service application,db/monitoring-for-ingest-service-application,/d/vJu8sgDnk/monitoring-for-ingest-service-app...,,dash-db,[Boson],False,0,True
598,2774,Okh3Z3H7k,Monitoring for Session Checker application,db/monitoring-for-session-checker-application,/d/Okh3Z3H7k/monitoring-for-session-checker-ap...,,dash-db,[Boson],False,0,True


In [76]:

print( f"res[meta] {res['meta']} \n ")
for key, item in res['meta'].items():
    print( key , res['meta'][key] ) 
print( ' \n \n \n ')   
print( f"res[dashboard] {res['dashboard']} \n ")
for key, item in res['dashboard'].items():
    print( key , ' - item  -  ', res['dashboard'][key] )


NameError: name 'res' is not defined

In [5]:
def get_failed_pods(**kwargs):
    # slownik= login_dict2() 
    auth=HTTPBasicAuth(
            BaseHook.get_connection("openshift_1").login,
            BaseHook.get_connection("openshift_1").password,
        )
    os.system(f'oc login https://api.devops.advantagedp.org:6443 -u {auth.username} -p {auth.password}')
    TOKEN = subprocess.run(['oc', 'whoami', '-t'], stdout=subprocess.PIPE, stderr=subprocess.PIPE) #stdout=subprocess.PIPE) 
    TOKEN = TOKEN.stdout.decode().strip()
    slownik = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        "namespace" : monitoring_failed_pods_data_pipeline[0]['name'],  #'data-pipeline-monitoring-prod',
        "app_name" : monitoring_failed_pods_data_pipeline[0]['app_name'],
        "token" : TOKEN,
        'url' :  monitoring_failed_pods_data_pipeline[0]['url'],  #make_endpoint('data-pipeline-monitoring-prod'),
        "headers" : {'Accept': 'application/json',"Authorization": "Bearer "+ TOKEN},
        "link" : 'https://console-openshift-console.apps.devops.advantagedp.org/k8s/ns/data-pipeline-monitoring-prod/pods',
    }
    # os.system('oc login https://api.devops.advantagedp.org:6443 -u devops_apps_pol-s -p k1.oandvaL3Q7ydJ')
    # proxies = {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'}
    # TOKEN = 'sha256~wfTI7SW-TKVccd4f_ZEMz2GIWmx2T3l0mOcGjx3CNR0'
    # headers = {'Accept': 'application/json',"Authorization": "Bearer "+ TOKEN}
    # url7 = make_endpoint('data-pipeline-monitoring-prod')
    # print( url7)
    res7 = requests.get(slownik['url'], headers=slownik['headers'], proxies=slownik['proxies'], verify=False)
    # res7 = requests.get(url7, headers=headers, proxies=proxies, verify=False)
    res_j = res7.json()
    df = pd.DataFrame.from_dict(res_j, orient='index')
    # res_j
    kind,apiVersion, metadata, items = zip(
        *(
            (
                res_j["kind"],
                res_j["apiVersion"],
                res_j["metadata"],
                res_j["items"],
            )
            for item in res_j
        )
    )
    df = pd.DataFrame(
        {
            "Kind": kind,
            "ApiVersion": apiVersion,
            "Metadata": metadata,
            "Items": items,
        }
    )
    # print(f'df \n {df.head(2)}' )

    # print( len(res_j['items']))
    # print( type(res_j['items'][0]))
    df2 = pd.DataFrame(res_j['items'])
    # print(f'\n df2 \n {df2.head()}' )
    now = dt.datetime.now()
    # czas = time.strftime()
    beauty_time = now.strftime("%Y-%m-%d  %H:%M:%S")
    

    name,namespace, creationTimestamp, manageFields_time, status_phase,status_starttime = zip(
        *(
            (
                df2.iloc[idx]['metadata']['name'],       
                df2.iloc[idx]['metadata']['namespace'],           
                df2.iloc[idx]['metadata']['managedFields'][0]['time'],# to samo   dt.datetime.fromisoformat(df2.iloc[idx]['metadata']['creationTimestamp'][:-1] + '+00:00').strftime("%Y-%m-%d  %H:%M:%S"),            
                df2.iloc[idx]['metadata']['managedFields'][0]['time'],# to samo             
                df2.iloc[idx]['status']['phase'],            
                df2.iloc[idx]['status']['startTime']
                
                # PONIZEJ BLAD PONIEWAZ df2.iloc[idx]['status']['containerStatuses'][0]['state'] BEDZIE # 3 {'running': {'startedAt': '2022-09-05T07:09:03Z'}}
                # 2 {'terminated': {'exitCode': 0, 'reason': 'Completed', 'startedAt': '2022-09-05T07:06:03Z', 'finishedAt': '2022-09-05T07:07:06Z', 'containerID': 'cri-o://c0cc76ca287fe25d23b02d8660c9092b328ee1baa07f1c6b7201f2ad144ee2fc'}}
                # df2.iloc[idx]['status']['containerStatuses'][0]['state']['terminated']['reason'],            
                # df2.iloc[idx]['status']['containerStatuses'][0]['state']['terminated']['startedAt'],
                # df2.iloc[idx]['status']['containerStatuses'][0]['state']['terminated']['finishedAt']
                
            )
            for idx in df2.index
        )
    )
    now = dt.datetime.now()
    # czas = time.strftime()
    beauty_time = now.strftime("%Y-%m-%d  %H:%M:%S")
    # print(f'now  {now}' )
    # print(f'beauty_time {beauty_time}' )
    
    df3 = pd.DataFrame(
        {
            "Name": name,
            "Namespace": namespace,
            "CreationTimestamp": creationTimestamp,
            # "ManageFields_time": manageFields_time,
            "Status_phase": status_phase,
            # "Status_startTime":status_starttime,
            "Time_now": dt.datetime.now().strftime("%Y-%m-%d  %H:%M:%S")
            # "Status_starttime": status_starttime,
            # "Status_containerStatuses_0_state_terminated_reason": status_containerStatuses_0_state_terminated_reason,
            # "Status_containerStatuses_0_state_terminated_startedAt": status_containerStatuses_0_state_terminated_startedAt,
            # "Status_containerStatuses_0_state_terminated_finishedAt": status_containerStatuses_0_state_terminated_finishedAt,
        }
    )
    # print( type(beauty_time))
    
    df3['App_name'] = df3['Name'].apply(lambda x : '-'.join(x.split('-')[:-2]))
    df3['Creation_time'] = df3['CreationTimestamp'].apply(change_creationtimestamp)
    df3['Date_diff_hours'] = df3['Creation_time'].apply(difference_dates_hours)
    df3['Date_diff_min'] = df3['Creation_time'].apply(difference_dates_minutes)
    df3['Link'] = df3['Name'].apply(lambda x : '/'.join([slownik['link'],x]))
    print(f'\n df3 \n {df3.iloc[0:]}' )
    print( df3['Link'].iloc[0])
    # print(f'\n df3 \n {df3.head()}' )
    error = 'Failed'
    errory = df3.loc[df3["Status_phase"]  ==  error ]
    output_errors = pd.DataFrame(errory, columns=['Name','Namespace','App_name','Status_phase','Creation_time','Date_diff_hours'])
    
    output_errors = output_errors[output_errors['Date_diff_hours'] <= 24]
    print(f'\n output_errors \n {output_errors.iloc[0:5]}' )
    # output_errors.sort_values(by=['Date_diff_hours'], ascending=False, inplace=True)
    # errory_male = errory.head()
    # print( f'\n \n Pods with status Failed  \n {errory}     ')
    output_errors_html = output_errors.to_html()
    

    running = 'Running'
    runningi = df3.loc[df3["Status_phase"]  ==  running ]
    print(f'\n PRZED  runningi \n {runningi.iloc[0:5]}' )
    runningi = runningi[runningi['Date_diff_hours'] <= 24]
    list_running_pods_longer_24 = runningi.index.value_counts()
    
    print( 'LISTA ',list_running_pods_longer_24)
    print(f'\n PO  runningi \n {runningi.iloc[0:5]}' )
    runnigni_html = runningi.to_html
    
    # if len(list_running_pods_longer_24) == 0:
    #     message = 'no long running pods'
    # else:
    #     message="something important to write if you need "
    # output_html = output_errors_html + \
    #     "<br/>{}<br/>".format(message)
        
    errory_html = errory.to_html
    # print(f'   ERRORY TO JSON \n {errory.to_json}' )
    # print( errory_html)
    # print( runnigni_html )
    errory_json = output_errors.to_json(orient="split")
    errory_json2 = errory.to_json(orient="split")
    parsed = json.loads(errory_json2)
    # print( json.dumps(parsed,indent=4))
    ####### !!!!!!!!!!!!!!!!!!!!!!!
    # kwargs["ti"].xcom_push(key="key", value=output_html)#errory_male_html)
    # PRZY RETURN WARTOSCI WYSKAKIWAL BLAD ZE METODA NIE JEST ITEROWALNA
    # ?
    # RETURN MUSI BYC NAZWA TASKA team_task_ CZYLI TAK JAK 
    # NAZWALES TASKA DO WYSYLANIA WIADOMOSCI 
    #return 'teams_task_'#errory_html
    return output_errors #return 'teams_task_'#errory_html

In [6]:
print(' JESTESM PLIKIEM devops_monitoring_FAILED_PODS ')
print( f' \n \n LONG RUNNING PODS \n \n ')

 JESTESM PLIKIEM devops_monitoring_FAILED_PODS 
 
 
 LONG RUNNING PODS 
 
 


In [7]:
def get_long_running_pods( long_time_running=6 , **kwargs):
    auth=HTTPBasicAuth(
            BaseHook.get_connection("openshift_1").login,
            BaseHook.get_connection("openshift_1").password,
        )
    os.system(f'oc login https://api.devops.advantagedp.org:6443 -u {auth.username} -p {auth.password}')
    TOKEN = subprocess.run(['oc', 'whoami', '-t'], stdout=subprocess.PIPE, stderr=subprocess.PIPE) #stdout=subprocess.PIPE) 
    TOKEN = TOKEN.stdout.decode().strip()
    slownik = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        "namespace" : monitoring_failed_pods_data_pipeline[0]['name'],  #'data-pipeline-monitoring-prod',
        "token" : TOKEN,
        'url' :  monitoring_failed_pods_data_pipeline[0]['url'],  #make_endpoint('data-pipeline-monitoring-prod'),
        "headers" : {'Accept': 'application/json',"Authorization": "Bearer "+ TOKEN}
    }
    # url7 = make_endpoint(NAMESPACE)
    # res7 = requests.get(url7, headers=headers, proxies=proxies , verify=False)
    res7 = requests.get(slownik['url'], headers=slownik['headers'], proxies=slownik['proxies'], verify=False)
    res_j = res7.json()
    df = pd.DataFrame.from_dict(res_j, orient='index')
    kind,apiVersion, metadata, items = zip(
        
        *(
            (
                res_j["kind"],
                res_j["apiVersion"],
                res_j["metadata"],
                res_j["items"],
            )
            for item in res_j
        )
    )
    df = pd.DataFrame(
        {
            "Kind": kind,
            "ApiVersion": apiVersion,
            "Metadata": metadata,
            "Items": items,
        }
    )
    # print(f' \t df \n {df.head()}' )
    # print( ' * ' * 20 )
    df2 = pd.DataFrame(res_j['items'])
    # print(f'\n df2 \n {df2.head(15)}' )
    now = dt.datetime.now()
    # czas = time.strftime()
    beauty_time = now.strftime("%Y-%m-%d  %H:%M:%S")
    
    name,namespace, creationTimestamp, manageFields_time, status_phase,status_starttime = zip(
        *(
            (
                df2.iloc[idx]['metadata']['name'],       
                df2.iloc[idx]['metadata']['namespace'],           
                df2.iloc[idx]['metadata']['creationTimestamp'],
                df2.iloc[idx]['metadata']['managedFields'][0]['time'],# to samo             
                df2.iloc[idx]['status']['phase'],            
                df2.iloc[idx]['status']['startTime']
                # datetime.fromisoformat(df2.iloc[idx]['metadata']['creationTimestamp'][:-1] + '+00:00').strftime("%Y-%m-%d  %H:%M:%S"),            
                # PONIZEJ BLAD PONIEWAZ df2.iloc[idx]['status']['containerStatuses'][0]['state'] BEDZIE # 3 {'running': {'startedAt': '2022-09-05T07:09:03Z'}}
                # 2 {'terminated': {'exitCode': 0, 'reason': 'Completed', 'startedAt': '2022-09-05T07:06:03Z', 'finishedAt': '2022-09-05T07:07:06Z', 'containerID': 'cri-o://c0cc76ca287fe25d23b02d8660c9092b328ee1baa07f1c6b7201f2ad144ee2fc'}}
                # df2.iloc[idx]['status']['containerStatuses'][0]['state']['terminated']['reason'],            
                # df2.iloc[idx]['status']['containerStatuses'][0]['state']['terminated']['startedAt'],
                # df2.iloc[idx]['status']['containerStatuses'][0]['state']['terminated']['finishedAt']
                
            )
            for idx in df2.index
        )
    )
    now = dt.datetime.now()
    # czas = time.strftime()
    beauty_time = now.strftime("%Y-%m-%d  %H:%M:%S")
    # print(f'now  {now}' )
    # print(f'beauty_time {beauty_time}' )
    
    df3 = pd.DataFrame(
        {
            "Name": name,
            "Namespace": namespace,
            "CreationTimestamp": creationTimestamp,
            # "ManageFields_time": manageFields_time,
            "Status_phase": status_phase,
            "Time_now": dt.datetime.now().strftime("%Y-%m-%d  %H:%M:%S"),
            # "Status_starttime": status_starttime,
            # "Status_containerStatuses_0_state_terminated_reason": status_containerStatuses_0_state_terminated_reason,
            # "Status_containerStatuses_0_state_terminated_startedAt": status_containerStatuses_0_state_terminated_startedAt,
            # "Status_containerStatuses_0_state_terminated_finishedAt": status_containerStatuses_0_state_terminated_finishedAt,
        }
    )
    
    
    df3['App_name'] = df3['Name'].apply(lambda x : '-'.join(x.split('-')[:-2]))
    df3['Creation_time'] = df3['CreationTimestamp'].apply(change_creationtimestamp)
    df3['Date_diff_hours'] = df3['Creation_time'].apply(difference_dates_hours)
    df3['Date_diff_min'] = df3['Creation_time'].apply(difference_dates_minutes)
    print( type(beauty_time))
    print(f'\n df3 \n {df3.iloc[0:50]}' )
    # print(f'\n  \t df3 \n {df3}' )
    error = 'Failed'
    errory = df3.loc[df3["Status_phase"]  ==  error ]

    errory_html = errory.to_html

    running = 'Running'
    runningi = df3.loc[df3["Status_phase"]  ==  running ]
    boson='dpm-dxc-boson'
    runningi = runningi.loc[runningi['Name'] != boson ]
    output_running = pd.DataFrame(runningi, columns=['Name','Namespace','App_name','Status_phase','Creation_time','Date_diff_min'])
    output_runnigni_html = output_running.to_html()
    message="something important to write if you need "
    output_html = output_runnigni_html + \
        "<br/>{}<br/>".format(message)
    

    errory_html = errory.to_html
    print( ' errory lista indexo ')
    print( errory.index.values )
    print( ' runningi lista indexo ')
    print( output_running.index.values )
    # kwargs["ti"].xcom_push(key="key", value=output_html)#errory_male_html)
    # PRZY RETURN WARTOSCI WYSKAKIWAL BLAD ZE METODA NIE JEST ITEROWALNA
    # ?
    # RETURN MUSI BYC NAZWA TASKA team_task_ CZYLI TAK JAK 
    # NAZWALES TASKA DO WYSYLANIA WIADOMOSCI 
    # return 'teams_task_' 
    return output_running

In [5]:
print( f' \n \n AIRFLOW_RESTAPI \n \n ')
# https://airflow.apache.org/docs/apache-airflow/stable/stable-rest-api-ref.html#operation/get_tasks
def airflow_restapi():
    # auth=HTTPBasicAuth(
    #         BaseHook.get_connection("openshift_1").login,
    #         BaseHook.get_connection("openshift_1").password,
    #     )
    auth = HTTPBasicAuth('ingest_pipeline-s', '1Rt.d5WssojT.2')

    slownik = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        # 'auth' : HTTPBasicAuth('ingest_pipeline-s', '1Rt.d5WssojT.2'),
        'url' :  "https://airflow-prod.apps.devops.advantagedp.org/api/experimental/dags/session_checker/dag_runs",
        # 'url' :  "https://airflow-prod.apps.devops.advantagedp.org/api/experimental/dags/data_pipeline/dag_runs",
        # 'url' :  "https://airflow-prod.apps.devops.advantagedp.org/api/experimental/dags/data_pipeline/tasks/sq-1665644773469_6e442987-b741-412f-a9d4-0183ad84cac7",
        # 'url' : "https://airflow-prod.dplapps.advantagedp.org/api/experimental/dags/ingest/dag_runs"
    }
    # url7 = make_endpoint(NAMESPACE)
    # res7 = requests.get(url7, headers=headers, proxies=proxies , verify=False)
    res7 = requests.get(slownik['url'], auth=auth, proxies=slownik['proxies'], verify=False)

    
    res_j = res7.json()
    # df = pd.DataFrame.from_dict(res_j, orient='index')
    print( 'ffffff \n \n ')
    print( res_j)
    print( type(res_j))
    for i, el in enumerate(res_j):
        print(i, el )
    dag_id,dag_run_url, execution_date, id,run_id, start_date,state = zip(
        *(
            (
                item["dag_id"],
                item["dag_run_url"],
                item["execution_date"],
                item["id"],
                item["run_id"],
                item["start_date"],
                item["state"],
            
                
            )
            for item in res_j
        )
    )
    df = pd.DataFrame(
        {
            "Dag_id" :  dag_id,
            "Dag_run_url" :  dag_run_url,
            "Execution_date" :  execution_date,
            "Id" :  id,
            "Run_id" :  run_id,
            "Start_date" :  start_date,
            "State" :  state,
        }
    )
    df['Execution_date_2'] = df['Execution_date'].apply(lambda x : dt.datetime.fromisoformat(x[:-1] + '+00:00') )
    print( ' ok ' )
    # return df
airflow_restapi()

 
 
 AIRFLOW_RESTAPI 
 
 


ConnectionError: SOCKSHTTPSConnectionPool(host='airflow-prod.dplapps.advantagedp.org', port=443): Max retries exceeded with url: /api/experimental/dags/ingest/dag_runs (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSHTTPSConnection object at 0x7fb2b56e3430>: Failed to establish a new connection: Connection closed unexpectedly'))

In [15]:
def last_success_date(dag_id, username, default_return_value='now-12h'):
    '''
    Takes dag_id and airflow username and using airflow api returns timedate of the last successful run in format accepted by the elasticsearch api
    default_return_value is optional and set for 12 hours behind. Value MUST be a proper elasticsearch date format, i.e. 'now-12h' or '2022-05-10T12:00:00' for more: https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-date-format.html
    '''
    endpoint_url = "https://airflow-prod.apps.devops.advantagedp.org/api/experimental/dags/"+dag_id+"/dag_runs"
    credentials = {
        "username": BaseHook.get_connection(username).login,
        "password": "z56fCgvBq6ECBomi",#BaseHook.get_connection(username).password,
    }

    try:
        response = get(endpoint_url, auth=(credentials.get(
            'username'), credentials.get('password')))
        if response.status_code != 200:
            raise Exception(
                "Airflow api returned !=200 status, raising exception")
    except Exception as err:
        print(err)
        raise AirflowException("Connection to the Airflow api failed") from err
    dag_runs_df = pd.DataFrame(json.loads(response.text))
    dag_runs_df = dag_runs_df[dag_runs_df.state == 'success']

    if dag_runs_df.empty:
        print('No previous successful runs, default: ' + default_return_value)
        return default_return_value
    else:
        last_success_run = dag_runs_df.iloc[-1:]
        start_date = last_success_run['start_date'].values[0]
        start_date = start_date[0:19]
        print('Last successful run date: '+start_date)
        return start_date

In [16]:
def last_success_date_epoch(dag_id, username='ingest_pipeline-s', default_return_hours=12, in_seconds=False):
    '''
    Takes dag_id and airflow username and using airflow api returns timedate of the last successful run in epoch miliseconds (seconds if in_seconds = True)
    IMPORTANT: default_return_hours is optional and set for 12 hours behind. Value describes how many hours substract from current date if no successful runs before has been found for this dag 
    OUTPUT: epoch time in miliseconds or seconds
    '''
    endpoint_url = "https://airflow-prod.apps.devops.advantagedp.org/api/experimental/dags/"+dag_id+"/dag_runs"
    credentials = {
        "username": 'ingest_pipeline-s',# BaseHook.get_connection(username).login,
        "password": '1Rt.d5WssojT.2',#BaseHook.get_connection(username).password,
    }
    ''' DODANE PRZEZE MNIE '''
    proxies = {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'}
    auth = HTTPBasicAuth('ingest_pipeline-s', '1Rt.d5WssojT.2')

    slownik = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        # 'auth' : HTTPBasicAuth('ingest_pipeline-s', '1Rt.d5WssojT.2'),
        'url' :  "https://airflow-prod.apps.devops.advantagedp.org/api/experimental/dags/session_checker/dag_runs",
    }
    res7 = requests.get(slownik['url'], auth=auth, proxies=slownik['proxies'], verify=False)
    ''' DODANE PRZEZE MNIE '''
    try:
        # response = get(endpoint_url, auth=(credentials.get(
        #     'username'), credentials.get('password')),proxies=proxies)
        response = requests.get(slownik['url'], auth=auth, proxies=slownik['proxies'], verify=False)
        if response.status_code != 200:
            raise Exception(
                "Airflow api returned !=200 status, raising exception")
    except Exception as err:
        print(err)
        raise AirflowException("Connection to the Airflow api failed") from err

    dag_runs_df = pd.DataFrame(json.loads(response.text))
    dag_runs_df = dag_runs_df[dag_runs_df.state == 'success']

    if dag_runs_df.empty:
        epoch_time = int(time.time() - 3600 * default_return_hours)
        if in_seconds == False:
            epoch_time = epoch_time * 1000

        print('No previous successful runs, default: ' + str(epoch_time))
        return epoch_time
    else:
        last_success_run = dag_runs_df.iloc[-1:]
        start_date = last_success_run['start_date'].values[0]
        start_date = parse(start_date)
        epoch_start_date = calendar.timegm(start_date.utctimetuple())
        print('Last successful run date: '+str(dt.datetime.utcfromtimestamp(
            epoch_start_date).strftime('%Y-%m-%d %H:%M:%S')))

        if in_seconds == False:
            epoch_start_date = epoch_start_date*1000
            print('Last successful run date in epoch miliseconds: ' +
                  str(epoch_start_date))
        else:
            print('Last successful run date in epoch seconds: ' +
                  str(epoch_start_date))

        return epoch_start_date



In [17]:
print( 'last_success_date_epoch', last_success_date_epoch('auto_labelling', 'ingest_pipeline-s'))

/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'airflow-prod.apps.devops.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/radekt/venv2airflow/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'airflow-prod.apps.devops.advantagedp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Last successful run date: 2022-09-16 13:14:07
Last successful run date in epoch miliseconds: 1663334047000
last_success_date_epoch 1663334047000


In [18]:
def get_yarn(**kwargs):
    # slownik= login_dict2() 
    # auth=HTTPBasicAuth(
    #         BaseHook.get_connection("yarn_1").login,
    #         BaseHook.get_connection("yarn_1").password,
    #     )
    url = 'https://mapr-web.advantagedp.org:20202/ws/v1/cluster/apps?user=ingest_pipeline-s&finalStatus=SUCCEEDED&startedTimeBegin=1568491200000'
    auth = HTTPBasicAuth('ingest_pipeline-s', '1Rt.d5WssojT.2')
    slownik = {
        "proxies" : {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'},
        "namespace" : monitoring_failed_pods_data_pipeline[0]['name'],  #'data-pipeline-monitoring-prod',
        "app_name" : monitoring_failed_pods_data_pipeline[0]['app_name'],
        'url' :  monitoring_failed_pods_data_pipeline[0]['url'],  #make_endpoint('data-pipeline-monitoring-prod'),
    }
    aaa = "https://mapr-web.advantagedp.org:20202"
    proxies = {'http': 'socks5h://192.168.67.2:1081', 'https': 'socks5h://192.168.67.2:1081'}
    res7 = requests.get(slownik['url'],  proxies=slownik['proxies'], verify=False)
    # res7 = requests.get(url7, headers=headers, proxies=proxies, verify=False)
    res_j = res7.json()
    df = pd.DataFrame.from_dict(res_j, orient='index')
    # res_j
    response = requests.get(url, auth=auth, proxies=proxies, verify=False)
    response_json = response.json()
    response2 = requests.get(url, auth=auth, proxies=proxies, verify=False)
    response2_json = response2.json()
    # print( response2_json)
    df1 = pd.DataFrame(response2_json)
    my_list = df1.columns.values.tolist()
    # print(my_list)
    # print( response2_json)
    df1_1 = df1['apps']['app']
    print( type(df1_1))
    # for el in df1_1:
    #     print( el, ' \n ' , type(el), ' \n ')
    print( df1_1[0].keys() )
    # print(len(df1_1))
    # names, ids, trackingUrls, diagnostics, queues = zip(
    #     *(
    #         (
    #             item["name"],
    #             item["id"],
    #             item["trackingUrl"],
    #             item["diagnostics"],
    #             item["queue"],
    #         )
    #         for item in response_json["apps"]["app"]
    #     )
    # )
    names, ids, users, names, queues, states, finalStatuss, progresss, trackingUIs, trackingUrls, diagnosticss, clusterIds, applicationTypes, applicationTagss, startedTimes, finishedTimes, elapsedTimes, amContainerLogss, amHostHttpAddresss, allocatedMBs, allocatedVCoress, runningContainerss, memorySecondss,vcoreSecondss, preemptedResourceMBs, preemptedResourceVCoress, numNonAMContainerPreempteds, numAMContainerPreempteds = zip(
        *(
            (
                item["name"],
                item["id"],
                item["user"],
                item["name"],
                item["queue"],
                item["state"],
                item["finalStatus"],
                item["progress"],
                item["trackingUI"],
                item["trackingUrl"],
                item["diagnostics"],
                item["clusterId"],
                item["applicationType"],
                item["applicationTags"],
                item["startedTime"],
                item["finishedTime"],
                item["elapsedTime"],
                item["amContainerLogs"],
                item["amHostHttpAddress"],
                item["allocatedMB"],
                item["allocatedVCores"],
                item["runningContainers"],
                item["memorySeconds"],
                item["vcoreSeconds"],
                item["preemptedResourceMB"],
                item["preemptedResourceVCores" ],
                item["numNonAMContainerPreempted"],
                item["numAMContainerPreempted"],
            )
            for item in response_json["apps"]["app"]
        )
    )
    df = pd.DataFrame(
        {
            "Names"  :names  ,      
            "Users" :users,
            "Ids" :  ids ,
            "Queues" :    queues ,
            "States" :     states,
            "FinalStatuss" :  finalStatuss ,    
            "Progresss" :  progresss , 
            "TrackingUIs" :   trackingUIs,
            "TrackingUrls" :   trackingUrls , 
            "Diagnosticss"  :   diagnosticss  ,  
            "ClusterIds" :    clusterIds,
            "ApplicationTypes" :   applicationTypes,
            "ApplicationTagss" :   applicationTagss , 
            "StartedTimes"  :    startedTimes ,
            "FinishedTimes"  :     finishedTimes  ,
            "ElapsedTimes" :    elapsedTimes,
            "AmContainerLogss" :   amContainerLogss,
            "AmHostHttpAddresss"  :    amHostHttpAddresss,
            "AllocatedMBs"  :    allocatedMBs,
            "AllocatedVCoress"  :   allocatedVCoress ,   
            "MemorySecondss" :    memorySecondss,
            "VcoreSecondss" :   vcoreSecondss,
            "PreemptedResourceMBs"  :    preemptedResourceMBs,
            "PreemptedResourceVCoress"  :     preemptedResourceVCoress , 
            "NumNonAMContainerPreempteds"  :  numNonAMContainerPreempteds,
            "NumAMContainerPreempteds" :   numAMContainerPreempteds,
        }
    )   
    print( df.head())

    return ':)'

In [19]:
# print( difference_dates_hours()   )     #   def difference_dates_hours(date1)   
# print( difference_dates_minutes() )     #   def difference_dates_minutes(date1)
# print( difference_dates_seconds() )     #   def difference_dates_seconds(date1) 
# print( change_creationtimestamp() )     #   def change_creationtimestamp(date)


# print( date_to_epoch()            )   #def date_to_epoch(date_input, in_seconds=False)
# print( get_failed_pods()          )   #def get_failed_pods(**kwargs)
# print( get_long_running_pods()    )   #def get_long_running_pods( long_time_running=6 , **kwargs)
# print( airflow_restapi()          )   #def airflow_restapi()
# print( last_success_date()        )   #def last_success_date(dag_id, username, default_return_value='now-12h')
# print( last_success_date_epoch()  )   # def last_success_date_epoch(dag_id, username='ingest_pipeline-s', default_return_hours=12, in_seconds=False)
# print( get_yarn()  )